<a href="https://colab.research.google.com/github/dpratap17/DeepLearningLab/blob/main/DL_Lab_Exp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/poems-100.csv')
df.head()


,text
0,"O my Luve's like a red, red rose\nThat’s newly..."
1,"The rose is red,\nThe violet's blue,\nSugar is..."
2,How do I love thee? Let me count the ways.\nI ...
3,"Had I the heavens' embroidered cloths,\nEnwrou..."
4,"I.\n Enough! we're tired, my heart and I.\n..."


In [2]:
print(df.columns)


Index(['text'], dtype='object')


In [3]:
text = " ".join(df['text'].astype(str)).lower()


In [4]:
import re

text = re.sub(r'\n', ' ', text)
text = re.sub(r'[^a-zA-Z\s]', '', text)
text = re.sub(r'\s+', ' ', text).strip()


In [5]:
print(text[:500])
print("Total words:", len(text.split()))


o my luves like a red red rose thats newly sprung in june o my luves like the melodie thats sweetly playd in tune as fair art thou my bonnie lass so deep in luve am i and i will luve thee still my dear till a the seas gang dry till a the seas gang dry my dear and the rocks melt wi the sun i will luve thee still my dear while the sands o life shall run and fare thee well my only luve and fare thee well a while and i will come again my luve tho it were ten thousand mile the rose is red the violets
Total words: 24676


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random


In [7]:
words = text.split()
vocab = sorted(set(words))

word_to_ix = {w:i for i,w in enumerate(vocab)}
ix_to_word = {i:w for w,i in word_to_ix.items()}

vocab_size = len(vocab)
print("Vocabulary size:", vocab_size)


Vocabulary size: 5439


In [8]:
def one_hot(idx, size):
    vec = np.zeros(size)
    vec[idx] = 1
    return vec


In [9]:
class RNN_Numpy:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size

        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, x, h_prev):
        h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h_prev) + self.bh)
        y = np.dot(self.Why, h) + self.by
        return h, y


In [10]:
hidden_size = 50
rnn = RNN_Numpy(vocab_size, hidden_size, vocab_size)

h_prev = np.zeros((hidden_size, 1))
x = one_hot(word_to_ix[words[0]], vocab_size).reshape(-1,1)

h, y = rnn.forward(x, h_prev)
print("Forward pass successful")


Forward pass successful


In [11]:
sequence_length = 5

data = []
targets = []

for i in range(len(words) - sequence_length):
    data.append(words[i:i+sequence_length])
    targets.append(words[i+sequence_length])


In [12]:
def to_one_hot_sequence(seq):
    return torch.tensor(
        [one_hot(word_to_ix[w], vocab_size) for w in seq],
        dtype=torch.float32
    )

X = torch.stack([to_one_hot_sequence(seq) for seq in data])
y = torch.tensor([word_to_ix[t] for t in targets])


/tmp/ipython-input-1498319719.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  return torch.tensor(


In [13]:
class RNN_OneHot(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [20]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64
dataset = TensorDataset(X_idx, y_idx)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

epochs = 40
for epoch in range(epochs):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        out = model_emb(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")



Epoch 1, Loss: 6.7226
Epoch 2, Loss: 6.0710
Epoch 3, Loss: 5.6453
Epoch 4, Loss: 5.2211
Epoch 5, Loss: 4.7994
Epoch 6, Loss: 4.3844
Epoch 7, Loss: 3.9842
Epoch 8, Loss: 3.6096
Epoch 9, Loss: 3.2683
Epoch 10, Loss: 2.9622
Epoch 11, Loss: 2.6861
Epoch 12, Loss: 2.4409
Epoch 13, Loss: 2.2188
Epoch 14, Loss: 2.0203
Epoch 15, Loss: 1.8364
Epoch 16, Loss: 1.6696
Epoch 17, Loss: 1.5174
Epoch 18, Loss: 1.3753
Epoch 19, Loss: 1.2449
Epoch 20, Loss: 1.1280
Epoch 21, Loss: 1.0171
Epoch 22, Loss: 0.9152
Epoch 23, Loss: 0.8259
Epoch 24, Loss: 0.7403
Epoch 25, Loss: 0.6631
Epoch 26, Loss: 0.5928
Epoch 27, Loss: 0.5286
Epoch 28, Loss: 0.4700
Epoch 29, Loss: 0.4175
Epoch 30, Loss: 0.3689
Epoch 31, Loss: 0.3264
Epoch 32, Loss: 0.2875
Epoch 33, Loss: 0.2526
Epoch 34, Loss: 0.2222
Epoch 35, Loss: 0.1934
Epoch 36, Loss: 0.1693
Epoch 37, Loss: 0.1488
Epoch 38, Loss: 0.1291
Epoch 39, Loss: 0.1123
Epoch 40, Loss: 0.0974


In [21]:
print(torch.min(y_idx), torch.max(y_idx))
print(len(set(y_idx.tolist())))
print(vocab_size)


tensor(0) tensor(5438)
5439
5439


In [22]:
y_fake = torch.randint(0, vocab_size, y_idx.shape)
out = model_emb(X_idx[:64])
loss = criterion(out, y_fake[:64])
print(loss.item())


21.807804107666016


In [23]:
for xb, yb in loader:
    optimizer.zero_grad()
    out = model_emb(xb)
    loss = criterion(out, yb)
    print("Batch loss:", loss.item())
    break


Batch loss: 0.05334652587771416


In [24]:
out = model_emb(X_idx[:64])
print(out.max().item(), out.min().item())


22.46104621887207 -17.693201065063477


In [15]:
def generate_text_onehot(start_words, length=20):
    model_oh.eval()
    words_generated = start_words[:]

    for _ in range(length):
        seq = words_generated[-sequence_length:]
        x = to_one_hot_sequence(seq).unsqueeze(0)
        with torch.no_grad():
            out = model_oh(x)
        next_word = ix_to_word[out.argmax().item()]
        words_generated.append(next_word)

    return " ".join(words_generated)

print(generate_text_onehot(words[:5]))


o my luves like a the the the the the the the the the the the the the the the the the the the the


In [16]:
X_idx = torch.tensor([[word_to_ix[w] for w in seq] for seq in data])
y_idx = torch.tensor([word_to_ix[t] for t in targets])


In [17]:
class RNN_Embedding(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [18]:
model_emb = RNN_Embedding(vocab_size, 50, 128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_emb.parameters(), lr=0.001)

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model_emb(X_idx)
    loss = criterion(outputs, y_idx)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 8.6277
Epoch 2, Loss: 8.5974
Epoch 3, Loss: 8.5667
Epoch 4, Loss: 8.5346
Epoch 5, Loss: 8.5003
Epoch 6, Loss: 8.4625
Epoch 7, Loss: 8.4197
Epoch 8, Loss: 8.3700
Epoch 9, Loss: 8.3113
Epoch 10, Loss: 8.2410


In [19]:
def generate_text_embedding(start_words, length=20):
    model_emb.eval()
    words_generated = start_words[:]

    for _ in range(length):
        seq = words_generated[-sequence_length:]
        x = torch.tensor([[word_to_ix[w] for w in seq]])
        with torch.no_grad():
            out = model_emb(x)
        next_word = ix_to_word[out.argmax().item()]
        words_generated.append(next_word)

    return " ".join(words_generated)

print(generate_text_embedding(words[:5]))


o my luves like a ring precious snowflakes ruts are gone of the just of the just of the just of the just of the
